## H-dence U-Net
---

参考
- 論文 [H-DenseUNet: Hybrid Densely Connected UNet for Liver and Tumor Segmentation from CT Volumes](https://arxiv.org/pdf/1709.07330.pdf)
- 公式 [GitHubリポジトリ](https://github.com/xmengli999/H-DenseUNet)

In [ ]:
!ls ../Preprocess/datasets/train-label/
!ls ../Preprocess/datasets/train-image/
!ls ../Preprocess/datasets/test-label/
!ls ../Preprocess/datasets/test-image/

In [1]:
def read_mhd_files(path):
    '''
    指定ディレクトリの mhd 形式のファイルを読み込み、
    1-1. 例の結合された配列 images の作成
    1-2. images の各スライスごとに症例番号・スライス番号を格納した image_filesの作成
    '''

    mhd_files = natsorted(glob.glob(path+'*.mhd'))

    # 1症例目のみ配列作成のため、単体で読み込み
    images = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_files[0]))
    image_files = []
    for i in range(images.shape[0]):
        image_files.append(mhd_files[0].split('/')[-1].split('.')[0] +'-'+str(i))    

    # 以降の症例
    for mhd_name in mhd_files:
        mhd_array = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(mhd_name))
        images = np.concatenate([images,mhd_array])
        for i in range(mhd_array.shape[0]):
            image_files.append(mhd_name.split('/')[-1].split('.')[0] +'-'+str(i))     

    # 0-1に正規化
    #images = (images-np.min(images)) / (np.max(images)/2) - 1

    return images #, image_files

In [2]:
import os
import cv2
import glob
import SimpleITK
import numpy as np
from natsort import natsorted
from matplotlib import pylab as plt
from skimage.util import random_noise
from tqdm import tqdm_notebook as tqdm


class dataProcess(object):

    def __init__(self, out_rows, out_cols, data_path  = "../Preprocess/datasets/train-image/",
                                           label_path = "../Preprocess/datasets/train-label/*/",
                                           test_path  = "../Preprocess/datasets/test-image/",
                                           testlabel_path = "../Preprocess/datasets/test-label/*/"):

        self.out_rows = out_rows
        self.out_cols = out_cols
        self.data_path = data_path
        self.label_path = label_path
        self.test_path = test_path
        self.testlabel_path = testlabel_path

    def _transform(self,imagein):
        image = imagein
        resize_image = cv2.resize(image, (512, 512), interpolation=cv2.INTER_NEAREST)
        if self.__channels and len(image.shape) < 3:  # make sure images are of shape(h,w,3)
            image = np.array([image for i in range(3)])
        else:
            resize_image = image
        return np.array(resize_image)

    def horizontal_flip(img):
        return img[:,::-1]

    def vertical_flip(img):
        return img[::-1,:]
        
    def upscale1_1_img(img):
        scale = 563     # floor(512*1.1)
        img = cv2.resize(img.astype(np.float64), (scale,scale), interpolation = cv2.INTER_NEAREST)
        img = img[25:537,25:537]
        img = img.astype(np.int32)
        return img

    def addnoise(img):
        mean = 0
        sigma = 50
        gauss = np.random.normal(mean,sigma,(512,512))
        img = img.astype(np.float64) + gauss
        img = img.astype(np.int32)
        return img
    
    def create_train_data(self):
        '''
        data_pathに存在する各症例を読み込み、rate倍のデータ拡張を行ったものを訓練データセットとする
        '''
        if not os.path.exists("trainImages.npy"):
            print('Creating training images...')

            imgdatas= read_mhd_files(self.data_path)
            num = len(imgdatas)
            rate = 4
            final_images = np.ndarray([num*rate,1,512,512],'int32')
            imglabels = read_mhd_files(self.label_path)
            final_label = np.ndarray([num*rate,1,512,512],'int32')
            for i in tqdm(range(num)):
                final_images[rate*i,0] = imgdatas[i]
                final_images[rate*i+1,0] = dataProcess.horizontal_flip(imgdatas[i])
                final_images[rate*i+2,0] = dataProcess.vertical_flip(imgdatas[i])
                final_images[rate*i+3,0] = dataProcess.upscale1_1_img(imgdatas[i])
                final_label[rate*i,0] = imglabels[i]
                final_label[rate*i+1,0] = dataProcess.horizontal_flip(imglabels[i])
                final_label[rate*i+2,0] = dataProcess.vertical_flip(imglabels[i])
                final_label[rate*i+3,0] = dataProcess.upscale1_1_img(imglabels[i])


            print(final_images.shape)
            print(final_label.shape)
            np.save("trainImages.npy",final_images)
            np.save("trainMasks.npy", final_label)

            #np.save("imgs_train.npy", imgdatas)
            #np.save("imgs_mask_train.npy", imglabels)
            print('Saving to train.npy files done.')

    def create_test_data(self):
        
        if not os.path.exists("testImages.npy"):
            print('Creating testing images...')

            imgdatas =  read_mhd_files(self.test_path)
            num = len(imgdatas)
            final_images = np.ndarray([num, 1, 512, 512],'int32')

            imglabels =  read_mhd_files(self.testlabel_path)
            final_label = np.ndarray([num, 1, 512, 512],'int32')
            for i in tqdm(range(num)):
                final_images[i, 0] = imgdatas[i]
                final_label[i, 0] = imglabels[i]
            print(final_images.shape)
            print(final_label.shape)
            np.save("testImages.npy", final_images)
            np.save("testMasks.npy", final_label)


if __name__ == "__main__":

    mydata = dataProcess(512,512)
    mydata.create_train_data()
    mydata.create_test_data()


Creating training images...



(11516, 1, 512, 512)
(11516, 1, 512, 512)
Saving to train.npy files done.
Creating testing images...



(386, 1, 512, 512)
(386, 1, 512, 512)


In [ ]:
from keras import backend as K

# GPU1つのみの設定
if 'tensorflow' == K.backend():
    import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))